In [1]:
import os
import json
from typing import Dict, List, Optional, Union, cast
import requests

from acquire import scrape_github_data
from prepare import words

from env import github_token, github_username

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [1]:
# Imports for modeling... and stuff

import pandas as pd
from prepare import basic_clean, tokenize, stem, lemmatize, remove_stopwords, prep_article_data, words
from acquire import scrape_github_data

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 250)

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import nltk
import re

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split

# Import Decision Tree and Random Forest ;D
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [4]:
#Run scraping function
# data = scrape_github_data()

In [2]:
#Turn scraped data into raw df
# df = pd.DataFrame(data)
df = pd.read_json('data.json')



#Check returned df 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   repo             500 non-null    object
 1   language         470 non-null    object
 2   readme_contents  500 non-null    object
dtypes: object(3)
memory usage: 11.8+ KB


In [3]:
df = words(df)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   repo                      500 non-null    object
 1   language                  500 non-null    object
 2   readme                    500 non-null    object
 3   clean                     500 non-null    object
 4   stemmed                   500 non-null    object
 5   lemmatized                500 non-null    object
 6   contains_python_keywords  500 non-null    int64 
 7   contains_cpp_keywords     500 non-null    int64 
 8   contains_js_keywords      500 non-null    int64 
dtypes: int64(3), object(6)
memory usage: 35.3+ KB


In [4]:
df.language.value_counts()

other         277
Python        104
JavaScript     78
C++            41
Name: language, dtype: int64

### Modeling prep

In [5]:
# Make copies of df with prepared columns and target
clean_df = df.copy()[['language', 'clean']]
stem_df = df.copy()[['language', 'stemmed']]
lem_df = df.copy()[['language', 'lemmatized']]


# Get splits for each of the above dfs and isolate target
X_clean = clean_df[['clean']]
y_clean = clean_df.language

X_clean_train, X_clean_test, y_clean_train, y_clean_test = train_test_split(X_clean, y_clean, test_size=.2, random_state=302)
X_clean_train, X_clean_validate, y_clean_train, y_clean_validate  = train_test_split(X_clean_train, y_clean_train, test_size=.3, random_state=302)

print(X_clean_train.shape, X_clean_validate.shape, X_clean_test.shape)

X_stem = stem_df[['stemmed']]
y_stem = stem_df.language

X_stem_train, X_stem_test, y_stem_train, y_stem_test = train_test_split(X_stem, y_stem, test_size=.2, random_state=302)
X_stem_train, X_stem_validate, y_stem_train, y_stem_validate  = train_test_split(X_stem_train, y_stem_train, test_size=.3, random_state=302)

print(X_stem_train.shape, X_stem_validate.shape, X_stem_test.shape)

X_lem = lem_df[['lemmatized']]
y_lem = lem_df.language

X_lem_train, X_lem_test, y_lem_train, y_lem_test = train_test_split(X_lem, y_lem, test_size=.2, random_state=302)
X_lem_train, X_lem_validate, y_lem_train, y_lem_validate  = train_test_split(X_lem_train, y_lem_train, test_size=.3, random_state=302)

print(X_lem_train.shape, X_lem_validate.shape, X_lem_test.shape)




(280, 1) (120, 1) (100, 1)
(280, 1) (120, 1) (100, 1)
(280, 1) (120, 1) (100, 1)


### Modeling

In [6]:
# "Clean" models
cv_clean = CountVectorizer()
tfidf_clean = TfidfVectorizer()

cv_clean_bow = cv_clean.fit_transform(X_clean_train.clean)

tf_clean_bow = tfidf_clean.fit_transform(X_clean_train.clean)


# Make and fit decision tree object for cv_clean_bow
cv_tree1 = DecisionTreeClassifier(max_depth=5)
cv_tree1.fit(cv_clean_bow, y_clean_train)

#Make and fit decision tree object for tf_clean_bow
tf_tree1 = DecisionTreeClassifier(max_depth=5)
tf_tree1.fit(tf_clean_bow, y_clean_train)

#Output tree scores
print(f'CV tree 1 score: {cv_tree1.score(cv_clean_bow, y_clean_train)}')
print(f'TF IDF tree 1 score: {tf_tree1.score(tf_clean_bow, y_clean_train)}')

CV tree 1 score: 0.825
TF IDF tree 1 score: 0.8464285714285714


In [7]:
# "Stemmed" models
cv_stem = CountVectorizer()
tfidf_stem = TfidfVectorizer()

# Bags
cv_stem_bow = cv_stem.fit_transform(X_stem_train.stemmed)
tf_stem_bow = tfidf_stem.fit_transform(X_stem_train.stemmed)

# Make and fit decision tree object for cv_stem_bow
cv_tree2 = DecisionTreeClassifier(max_depth=5)
cv_tree2.fit(cv_stem_bow, y_stem_train)

#Make and fit decision tree object for tf_stem_bow
tf_tree2 = DecisionTreeClassifier(max_depth=5)
tf_tree2.fit(tf_stem_bow, y_stem_train)


#Get tree score
print(f'CV tree 2 score: {cv_tree2.score(cv_stem_bow, y_stem_train)}')
#Get tree score
print(f'TF IDF tree 2 score: {tf_tree2.score(tf_stem_bow, y_stem_train)}')

CV tree 2 score: 0.8214285714285714
TF IDF tree 2 score: 0.85


In [8]:
# "Lemmatized" models
cv_lem = CountVectorizer()
tfidf_lem = TfidfVectorizer()

cv_lem_bow = cv_lem.fit_transform(X_lem_train.lemmatized)
tf_lem_bow = tfidf_lem.fit_transform(X_lem_train.lemmatized)

# Make and fit decision tree object for cv_lem_bow
cv_tree3 = DecisionTreeClassifier(max_depth=5)
cv_tree3.fit(cv_lem_bow, y_lem_train)

#Make and fit decision tree object for tf_lem_bow
tf_tree3 = DecisionTreeClassifier(max_depth=5)
tf_tree3.fit(tf_lem_bow, y_lem_train)

# Output tree scores
print(f'CV tree score: {cv_tree3.score(cv_lem_bow, y_lem_train)}') 
print(f'TFIDF tree score: {tf_tree3.score(tf_lem_bow, y_lem_train)}')

CV tree score: 0.825
TFIDF tree score: 0.8392857142857143


In [9]:
dec_tree_training_scores= {
    'CV_clean': cv_tree1.score(cv_clean_bow, y_clean_train),
    'CV_stem': cv_tree2.score(cv_stem_bow, y_stem_train),
    'CV_lem': cv_tree3.score(cv_lem_bow, y_lem_train),
    'TF/IDF_clean': tf_tree1.score(tf_clean_bow, y_clean_train),
    'TF/IDF_stem': tf_tree2.score(tf_stem_bow, y_stem_train),
    'TF/IDF_lem': tf_tree3.score(tf_lem_bow, y_lem_train)
}

dec_tree_training_scores
#pd.DataFrame(dec_tree_training_scores, index=)

{'CV_clean': 0.825,
 'CV_stem': 0.8214285714285714,
 'CV_lem': 0.825,
 'TF/IDF_clean': 0.8464285714285714,
 'TF/IDF_stem': 0.85,
 'TF/IDF_lem': 0.8392857142857143}

### Takeaways: Both vectorizers had scored the same on clean, stemmed, and lemmatized preparations of README text in previous evaluations. Current version of text preparations have models using TF/IDF vectorizers scoring higher on train split.

### Update (16-May-2022): Data increased from 100 to 500 records. TF/IDF vectorizers still scoring higher.

In [10]:
print('TF/IDF Decision Tree scores on validate split:')
print('----------------------------------------------')
#Transform validate split with vectorizer
tf_clean_bow_val = tfidf_clean.transform(X_clean_validate.clean)

#Output tf_tree1 score on validate
print(f'Tree 1 [clean] score: {tf_tree1.score(tf_clean_bow_val, y_clean_validate)}')

#Transform validate split with vectorizer
tf_stem_bow_val = tfidf_stem.transform(X_stem_validate.stemmed)

#Output tf_tree2 score on validate
print(f'Tree 2 [stemmed] score: {tf_tree2.score(tf_stem_bow_val, y_stem_validate)}')

#Transform validate split with vectorizer
tf_lem_bow_val = tfidf_lem.transform(X_lem_validate.lemmatized)

#Output tf_tree3 score on validate
print(f'Tree 3 [lemmatized] score: {tf_tree3.score(tf_lem_bow_val, y_lem_validate)}')

TF/IDF Decision Tree scores on validate split:
----------------------------------------------
Tree 1 [clean] score: 0.75
Tree 2 [stemmed] score: 0.75
Tree 3 [lemmatized] score: 0.7666666666666667


### Random Forest models, 100 estimators, max depth 15

In [11]:
# Make vectorizer objects
cv_clean2 = CountVectorizer()
tfidf_clean2 = TfidfVectorizer()

#Fit vectorizer objects
cv_clean2_bow = cv_clean2.fit_transform(X_clean_train.clean)
tf_clean2_bow = tfidf_clean2.fit_transform(X_clean_train.clean)

# Make and fit random forest object for cv_clean2_bow
cv_rf1 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf1.fit(cv_clean2_bow, y_clean_train)

#Make and fit random forest object for tf_clean2_bow
tf_rf1 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf1.fit(tf_clean2_bow, y_clean_train)

#Output rf scores
print(f'CV RF 1 score: {cv_rf1.score(cv_clean2_bow, y_clean_train)}')
print(f'TF IDF RF 1 score: {tf_rf1.score(tf_clean2_bow, y_clean_train)}')

# Make vectorizer objects
cv_stem2 = CountVectorizer()
tfidf_stem2 = TfidfVectorizer()

#Fit vectorizer objects
cv_stem2_bow = cv_stem2.fit_transform(X_stem_train.stemmed)
tf_stem2_bow = tfidf_stem2.fit_transform(X_stem_train.stemmed)

# Make and fit random object for cv_stem2_bow
cv_rf2 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf2.fit(cv_stem2_bow, y_stem_train)

#Make and fit decision rf object for tf_stem2_bow
tf_rf2 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf2.fit(tf_stem2_bow, y_stem_train)

#Output rf scores
print(f'CV RF 2 score: {cv_rf2.score(cv_stem2_bow, y_stem_train)}')
print(f'TF IDF RF 2 score: {tf_rf2.score(tf_stem2_bow, y_stem_train)}')
# Make vectorizer objects
cv_lem2 = CountVectorizer()
tfidf_lem2 = TfidfVectorizer()

#Fit vectorizer objects
cv_lem2_bow = cv_lem2.fit_transform(X_lem_train.lemmatized)
tf_lem2_bow = tfidf_lem2.fit_transform(X_lem_train.lemmatized)

# Make and fit decision tree object for cv_lem2_bow
cv_rf3 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf3.fit(cv_lem2_bow, y_lem_train)

#Make and fit decision rf object for tf_lem2_bow
tf_rf3 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf3.fit(tf_lem2_bow, y_lem_train)

#Output rf scores
print(f'CV RF 3 score: {cv_rf3.score(cv_lem2_bow, y_lem_train)}')
print(f'TF IDF RF 3 score: {tf_rf3.score(tf_lem2_bow, y_lem_train)}')

CV RF 1 score: 0.8035714285714286
TF IDF RF 1 score: 0.8428571428571429
CV RF 2 score: 0.8285714285714286
TF IDF RF 2 score: 0.8821428571428571
CV RF 3 score: 0.8035714285714286
TF IDF RF 3 score: 0.85


### Both vectorizers scoring the same on training split for all text preparations. All move on to validate.

### Update (16-May-2022 11:15) TF/IDF vectorizers score higher than count vectorizers.

In [19]:
# Transform count vectorizers on validate
#cv_clean2_bow_val = cv_clean2.transform(X_clean_validate.clean)
#cv_stem2_bow_val = cv_stem2.transform(X_stem_validate.stemmed)
#cv_lem2_bow_val = cv_lem2.transform(X_lem_validate.lemmatized)

# Output CV RF scores
#print(f'CV RF 1 score on validate: {cv_rf1.score(cv_clean2_bow_val, y_clean_validate)}')
#print(f'CV RF 2 score on validate: {cv_rf2.score(cv_stem2_bow_val, y_stem_validate)}')
#print(f'CV RF 3 score on validate: {cv_rf3.score(cv_lem2_bow_val, y_lem_validate)}')

# Transform TF IDF vectorizers on validate
tf_clean2_bow_val = tfidf_clean2.transform(X_clean_validate.clean)
tf_stem2_bow_val = tfidf_stem2.transform(X_stem_validate.stemmed)
tf_lem2_bow_val = tfidf_lem2.transform(X_lem_validate.lemmatized)

# Output TF IDF RF scores
print(f'TF IDF RF 1 score on validate: {tf_rf1.score(tf_clean2_bow_val, y_clean_validate)}')
print(f'TF IDF RF 2 score on validate: {tf_rf2.score(tf_stem2_bow_val, y_stem_validate)}')
print(f'TF IDF RF 3 score on validate: {tf_rf3.score(tf_lem2_bow_val, y_lem_validate)}')

TF IDF RF 1 score on validate: 0.675
TF IDF RF 2 score on validate: 0.7
TF IDF RF 3 score on validate: 0.675


### Random Forest model using TF/IDF vectorizer on stemmed text performing at 70% accuracy on validate.

### Make functions to speed up modeling process

In [62]:
#def get_cv_dt_score (text_data, target, max_depth, ngram_range=False, train_split=True):
    #'''
    #Takes in text_data, a target variable, an optional ngram range argument (min_n, max_n) for a Count Vectorizer object, and a max_depth kwarg for a Decision Tree,
    #and returns a bag of words (bow), the Decision Tree object and its accuracy score on the train split.
    #'''
    #cv = CountVectorizer(ngram_range)
    #bow = cv.fit_transform(text_data)
    #tree = DecisionTreeClassifier(max_depth=max_depth)
    #tree.fit(bow, target)
    
    #if train_split:
        #dt_score = tree.score(bow, target)
        #return tree, bow, dt_score
    #else:
        #print('Use tree and bow returned from train split to get score on validate/test split.')
    


In [33]:
#def get_tfidf_dt_score (text_data, target, depth, train_split=True):
    #'''
    #Takes in text_data, a target variable, and a depth(max_depth) kwarg for a Decision Tree model,
    #and returns the Decision Tree object and an accompanying accuracy score for the model.
    #'''
    #tfidf = TfidfVectorizer()
    #bow = tfidf.fit_transform(text_data)
    #tree = DecisionTreeClassifier(max_depth=depth)
    #tree.fit(bow, target)

    #if train_split:
        #dt_score = tree.score(bow, target)
    #else:
        #raise Exception('')

    #return tree, dt_score


In [ ]:
#def get_cv_rf_score(text_data, target, depth, n_estimators, train_split=True):
    #'''
    #Takes in text data, a target variable, depth and n_estimators kwargs for a Random Forest model,
    #and returns the Random Forest object and an accompanying accuracy score for the model.
    #'''
    #cv = CountVectorizer()
    #bow = cv.fit_transform(text_data)
    #rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=depth)
    #rf.fit(bow, target)
    
    #if train_split:
        #rf_score = rf.score(bow, target)
    #else:
        #bow = cv.transform(text_data)
        #rf_score = rf.score(bow, target)
    
    #return rf, rf_score

### Bags of bigrams -- Dec trees

In [13]:
# Bigrams

# Make bags
cv_clean3 = CountVectorizer(ngram_range=(2,2))
tfidf_clean3 = TfidfVectorizer(ngram_range=(2,2))

# Fill bags with words
cv_clean3_bow = cv_clean3.fit_transform(X_clean_train.clean)
tf_clean3_bow = tfidf_clean3.fit_transform(X_clean_train.clean)


# Make and fit decision tree object for cv_clean3_bow
cv_tree4 = DecisionTreeClassifier(max_depth=5)
cv_tree4.fit(cv_clean3_bow, y_clean_train)

#Make and fit decision tree object for tf_clean_bow
tf_tree4 = DecisionTreeClassifier(max_depth=5)
tf_tree4.fit(tf_clean3_bow, y_clean_train)

#Output tree scores
print(f'CV tree 4 score: {cv_tree4.score(cv_clean3_bow, y_clean_train)}')
print(f'TF/IDF tree 4 score: {tf_tree4.score(tf_clean3_bow, y_clean_train)}')

CV tree 4 score: 0.7214285714285714
TF IDF tree 4 score: 0.725


In [14]:
# Make bags
cv_stem3 = CountVectorizer(ngram_range=(2,2))
tfidf_stem3 = TfidfVectorizer(ngram_range=(2,2))

# Fill bags with words
cv_stem3_bow = cv_stem3.fit_transform(X_stem_train.stemmed)
tf_stem3_bow = tfidf_stem3.fit_transform(X_stem_train.stemmed)


# Make and fit decision tree object for cv_stem3_bow
cv_tree5 = DecisionTreeClassifier(max_depth=5)
cv_tree5.fit(cv_stem3_bow, y_stem_train)

#Make and fit decision tree object for tf_stem_bow
tf_tree5 = DecisionTreeClassifier(max_depth=5)
tf_tree5.fit(tf_stem3_bow, y_stem_train)

#Output tree scores
print(f'CV tree 5 score: {cv_tree5.score(cv_stem3_bow, y_stem_train)}')
print(f'TF/IDF tree 5 score: {tf_tree5.score(tf_stem3_bow, y_stem_train)}')

CV tree 5 score: 0.725
TF IDF tree 5 score: 0.7285714285714285


In [15]:
cv_lem3 = CountVectorizer(ngram_range=(2,2))
tfidf_lem3 = TfidfVectorizer(ngram_range=(2,2))

# Fill bags with words
cv_lem3_bow = cv_lem3.fit_transform(X_lem_train.lemmatized)
tf_lem3_bow = tfidf_lem3.fit_transform(X_lem_train.lemmatized)


# Make and fit decision tree object for cv_lem3_bow
cv_tree6 = DecisionTreeClassifier(max_depth=5)
cv_tree6.fit(cv_lem3_bow, y_lem_train)

#Make and fit decision tree object for tf_lem_bow
tf_tree6 = DecisionTreeClassifier(max_depth=5)
tf_tree6.fit(tf_lem3_bow, y_lem_train)

#Output tree scores
print(f'CV tree 6 score: {cv_tree6.score(cv_lem3_bow, y_lem_train)}')
print(f'TF/IDF tree 6 score: {tf_tree6.score(tf_lem3_bow, y_lem_train)}')

CV tree 6 score: 0.7214285714285714
TF IDF tree 6 score: 0.725


In [23]:
print('TF/IDF [Bigram bags] Decision Tree scores on validate split:')
print('------------------------------------------------------------')
#Transform validate split with vectorizer
tf_clean3_bow_val = tfidf_clean3.transform(X_clean_validate.clean)

#Output tf_tree1 score on validate
print(f'Tree 4 [clean] score: {tf_tree4.score(tf_clean3_bow_val, y_clean_validate)}')

#Transform validate split with vectorizer
tf_stem3_bow_val = tfidf_stem3.transform(X_stem_validate.stemmed)

#Output tf_tree2 score on validate
print(f'Tree 2 [stemmed] score: {tf_tree5.score(tf_stem3_bow_val, y_stem_validate)}')

#Transform validate split with vectorizer
tf_lem3_bow_val = tfidf_lem3.transform(X_lem_validate.lemmatized)

#Output tf_tree3 score on validate
print(f'Tree 3 [lemmatized] score: {tf_tree6.score(tf_lem3_bow_val, y_lem_validate)}')

TF/IDF [Bigram bags] Decision Tree scores on validate split:
------------------------------------------------------------
Tree 4 [clean] score: 0.7666666666666667
Tree 2 [stemmed] score: 0.7583333333333333
Tree 3 [lemmatized] score: 0.7583333333333333


### Bigram Random Forests

In [20]:
# Make vectorizer objects
cv_clean4 = CountVectorizer(ngram_range=(2,2))
tfidf_clean4 = TfidfVectorizer(ngram_range=(2,2))

#Fit vectorizer objects
cv_clean4_bow = cv_clean4.fit_transform(X_clean_train.clean)
tf_clean4_bow = tfidf_clean4.fit_transform(X_clean_train.clean)

# Make and fit random forest object for cv_clean4_bow
cv_rf4 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf4.fit(cv_clean4_bow, y_clean_train)

#Make and fit random forest object for tf_clean4_bow
tf_rf4 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf4.fit(tf_clean4_bow, y_clean_train)

#Output rf scores
print(f'CV RF 4 score: {cv_rf4.score(cv_clean4_bow, y_clean_train)}')
print(f'TF IDF RF 4 score: {tf_rf4.score(tf_clean4_bow, y_clean_train)}')

# Make vectorizer objects
cv_stem4 = CountVectorizer()
tfidf_stem4 = TfidfVectorizer()

#Fit vectorizer objects
cv_stem4_bow = cv_stem4.fit_transform(X_stem_train.stemmed)
tf_stem4_bow = tfidf_stem4.fit_transform(X_stem_train.stemmed)

# Make and fit random object for cv_stem4_bow
cv_rf5 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf5.fit(cv_stem4_bow, y_stem_train)

#Make and fit decision rf object for tf_stem4_bow
tf_rf5 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf5.fit(tf_stem4_bow, y_stem_train)

#Output rf scores
print(f'CV RF 2 score: {cv_rf5.score(cv_stem4_bow, y_stem_train)}')
print(f'TF IDF RF 2 score: {tf_rf5.score(tf_stem4_bow, y_stem_train)}')
# Make vectorizer objects
cv_lem4 = CountVectorizer()
tfidf_lem4 = TfidfVectorizer()

#Fit vectorizer objects
cv_lem4_bow = cv_lem4.fit_transform(X_lem_train.lemmatized)
tf_lem4_bow = tfidf_lem4.fit_transform(X_lem_train.lemmatized)

# Make and fit decision tree object for cv_lem4_bow
cv_rf6 = RandomForestClassifier(n_estimators= 100, max_depth=15)
cv_rf6.fit(cv_lem4_bow, y_lem_train)

#Make and fit decision rf object for tf_lem4_bow
tf_rf6 = RandomForestClassifier(n_estimators=100, max_depth=15)
tf_rf6.fit(tf_lem4_bow, y_lem_train)

#Output rf scores
print(f'CV RF 3 score: {cv_rf6.score(cv_lem4_bow, y_lem_train)}')
print(f'TF IDF RF 3 score: {tf_rf6.score(tf_lem4_bow, y_lem_train)}')

CV RF 4 score: 0.5785714285714286
TF IDF RF 4 score: 0.5892857142857143
CV RF 2 score: 0.8321428571428572
TF IDF RF 2 score: 0.8821428571428571
CV RF 3 score: 0.7964285714285714
TF IDF RF 3 score: 0.8464285714285714


### TF/IDF vectorizers outperform count vectorizers overall. RFs on stemmed bigrams scoring highest.

In [24]:
# Transform count vectorizers on validate
cv_stem4_bow_val = cv_stem4.transform(X_stem_validate.stemmed)
cv_lem4_bow_val = cv_lem4.transform(X_lem_validate.lemmatized)

# Transform TF/IDF vectorizers on validate
tf_stem4_bow_val = tfidf_stem4.transform(X_stem_validate.stemmed)
tf_lem4_bow_val = tfidf_lem4.transform(X_lem_validate.lemmatized)

#Output CV RF 5 & 6 scores on validate
print(f'CV RF 5 score on validate: {cv_rf5.score(cv_stem4_bow_val, y_stem_validate)}')
print(f'CV RF 6 score on validate: {cv_rf6.score(cv_lem4_bow_val, y_lem_validate)}')

# Output TF/IDF RF 5 & 6 scores on validate
print(f'TF/IDF RF 5 score on validate: {tf_rf5.score(tf_stem4_bow_val, y_stem_validate)}')
print(f'TF/IDF RF 6 score on validate: {tf_rf6.score(tf_lem4_bow_val, y_lem_validate)}')

CV RF 5 score on validate: 0.675
CV RF 6 score on validate: 0.6666666666666666
TF/IDF RF 5 score on validate: 0.675
TF/IDF RF 6 score on validate: 0.6666666666666666
